## lessaib 5 Notes

- Inputs
- Weights/parameters
    - Random
- Activations
- Activation functions / nonlinearities
- Output
- Loss
- Metric
- Cross-entropy
- Softmax
- Fine tuning
    - Layer deletion and random weights
    - Freezing & unfreezing

![a](lesson5/lesson5-1.png "")

### Inputs

It is the features.
In this case, MNIST input is handwriting pictures of 28 by 28 pixels. Sine we use each pixel as a feature, so the input shape will be 28 * 28 = 784


It is one colume of 784 rows.

If it is a color picture, which mean it may have 3 channel, so the input feature will be 28 * 28 * 3


### Weights

For matrix multiplication to happen correctly, one dimention of shape has to match, so the row of the weighs is fixed to the row of inputs, which is 784

Note: the input layer is probably transform into shape of [1, 784]

The other dimention is you pick really. Depends on want you want to outcome activation is, can be 1, can be 5, 10, 1 million. In this picture, we picked 5.

That's the part what a neural networks is, how many neurons you want in this layer.

Note: the last activation, the output, should be the dimention you need. For MNIST, we want to predict number from 0 to 10, so we will have dimention shape of [ 10, 1 ]. So we should pick 10 at the last layer.

This weights is initialized as random number, but also should be normalized. Check out kaming normal

### Activation

The new layer of activation is the result of a matrix multication between last activations and weights is performed.


![a](lesson5/lesson5-2.png "")

### Activation Function

This is a function that modify each value in the activation. ( element-wise function )

ReLU repalce every negative number to zero.

### Repeat next layer

The process is repeat next layer, where having a new set of weights, this time is 8.

Just remember that the last layer should be 10, for the example. So your last layer of weights should second-last len(activation) and 10 column

### Layer

It is just steps, can be a step of activation functions, can be a matrix multication of another weights

![a](lesson5/lesson5-3.png "")

### Last Layer

This is simply the output, which is a activation. 

But most of the time, we can also have another activation function, sometimes a sigmoid function, the calculate the probability of the output. 

E.g. the output layer has these output for MNIST by predicting what number of this handwriting should be:

```
[[0.123], [0.232], [0.897], [0.123], [0.123], [0.232], [0.456], [0.123], [0.232], [0.345], [0.254]]
```

See that the 2nd value is high, so the neural network has predicted that the handwriting should be 2

We use sigmoid function to 'cap' the output. 

## Lost Function

It is a function to calculate how accurate the network predict.

### Mean Squred Error

For Mean-Squred Error, it is just Sum of (y-y_hat)^2

## Universal approximation theorem

It is basically saying if there are big-enough data, for a enough complex of function ( think of it as a huge multi-layer neural net ), you can solve any arbitrariy high level of accuracy for that function ( the basis of machine learning ). One important thing is that we need to give some non-linearity to these functions.

Otherwise, this complex of functions which is a linear-function is just a equation, it will give a result, but can not be use for new data. ( there is no exact answer to a problem, can be very close, but never 100% )

Why use ReLU? 

ReLU makes the activations becomes non-linearity. Think of it as randomness. Each time weights and input is randomized, and ReLU remove some of the features of a function in these functions.

Andrew Ng's Machine Learning course give a much more cleared reason on why this works, with support of Math.



## Back Propagation

Just remember it is just:

weights(i-1) -= weights(i).gradient * learning_rate

, where i is the layer

## Transfer Learning

For a specific model, like Resnet, we can use pre-trained model, to train a new problem, which use the same structure of model.

For example, ImageNet has a trained resnet-52 model, which classify 1000 things.

We can download the cache of that model, and use that to train a new same structure of resnet-52 model, which classify 3 type of bears.

The process is called Fine-Tuning

![a](lesson5/lesson5-4.png "")

## Fine-Tuning

Resnet is basically a CNN. One feature that CNN has is that for the first 1-3 layers, it is very good at finding image features. E.g. edges, shapes, lines...

The deeper the layer it goes, the more complex idea of a image feature it will recognize. At 10th layer, it maybe good at recognize idea like eye-balls, furry-stuff, metal-shape, etc.

The idea of fine-tuning is that, we do not need to re-train the first few layers, just remove the last few layers that recognize complex idea that specific to classifying 1000 stuff, and re-train the last few layers with new 3 bear classify's data.

For example, Freeze first 2/3 of the layers, and re-init the last 1/3 of layers, and only train the last 1/3 of layers by only back-propagate into those layers.


![a](lesson5/lesson5-5.png "")

![a](lesson5/lesson5-6.png "")

![a](lesson5/lesson5-7.png "")

![a](lesson5/lesson5-8.png "")

## Discriminative Learning Rate

In Fine-Tuning, after training only 1/3 of the last layers, we can un-freeze the first 2/3 of layers. But we do not want to have the first 2/3 of the layers have the same learning rate of 1/3 of layers, because those layers should be pretty much optimized already. We can actually make first 2/3 of layers worse.

So we set the learning rate of first 2/3 of layers to be smaller.

E.g. first 1/3 of layers has learning rate of 1e-5, 2nd of 1/3 layers has 1e-4, last has 1e-3

In fastai:
```
model.fit( 1, 1e-3 )  # No discriminative LR
model.fit( 1, slice(1e-3) ) # deafult 2/3 layers is one-third of last 1/3 of layers
model.fit( 1, slice(1e-5, 1e-3) ) # 1st 1/3 layer is 1e-5, 2nd is 1e-4, 3rd is 1e-3
```

## Affine Function

it is a linear function.

If we just put a affine function on top of another affine function, it is still just an affine function. So we need to give some non-linearity to it. like a relu.

## Weight Decay

![a](lesson5/lesson5-9.png "")

To overcome overfitting, we can use weight decay.
We basically penalize complexity by putting sum of all weights into the result of loss as well. But because some weights is negative, so we need to use square of each weights.

![a](lesson5/lesson5-10.png "")
```
Loss = MSE(y_hat, y) + wd * sum(w^2)
```
Weight decay (WD) is to make sure the second part of the equation become too huge, making all weights to be zero when training.

fastai has a good result when wd = 0.1, but default to 0.01

```
learn = collab_learner(data, n_factors=40, y_range=[0, 5.5], wd=1e-1)
```

### Check weight decay in lesson5-sgd
```
# weight decay
    w2 = 0.
    for p in model.parameters(): w2 += (p**2).sum()
    # add to regular loss
    loss = loss_func(y_hat, y) + w2*wd
```

### Why called Weight Decay?

![a](lesson5/lesson5-11.png "")

The gradient function L(x,w), base on train rule, we take derivatives on both loss function and wd * w^2 separately. the derivatives of wd * w^2 is 2 * wd * w, which is a negative number. Hence the "Decay" comes from.

All weight decay does is to subtracts some constant times to weights every time we do a batch.


## L2 regularization

the wd * w^2 part is L2 regularization. While weight decay is after derivatives part ( 2 * wd * w). So, they are mathematically the same. ( In this case )

### Check graddesc.xlsm

This is the excel version of performing gradient decent. It also have different loss functions methods.

## Finite Differencing

To calculate gradient, we can use derivative, but we can also use finite differencing.

You can just add a very small number to your value e as e_hat, that compare the diference between e and e_hat. The slope of e and e_hat is the de/db.

![a](lesson5/lesson5-12.png "")
![a](lesson5/lesson5-13.png "")

The 0.01 in the excel is the small number

## Momentum

It is to speed up the converging process.

![a](lesson5/lesson5-14.png)

For the gradient of this loop, we will now have this formula:
```
gradient(i) = gradient(i-1) * 0.9 + gradient(i) * 0.1
```
So, if the last gradient has the same direction of this gradient, it will speed up; while if we overshoot and the current gradient is the opposite direction, it will slow down. It will eventually go back to the right direction.

0.9 and 0.1 is your choice

It also works on this case as well:
![](lesson5/lesson5-15.png)

![](lesson5/lesson5-16.png)
This is also called exponentially weighted moving average.

```
opt = optim.SGD(model.parameters(), lr, momentum=0.9)
```

## RMSProp

TODO

rmpsprop is very similar to momentum, but the gradient is squared.
the value of rmsprop is small if the gradient is small and steady.
the value of rmsprop is big if the gradient is big or it is highly volatile.

Maybe wrong
weights - lr * gradient / sqrt(rmsprop)

![](lesson5/lesson5-17.png)

## Adam

TODO

Adam is momentum + RMSProp

## fit one cycle

fastai does this for us, but basically:

at the beginning of training, we do not know what direction to go, so we should have a low learning rate. 

but once in the middle, we know where to go, we can have a higher learning rate. 

In the end, we need to anneal the learning rate by lowering it, so we can converge.

Momentum is opposite, if in the beginning we are going the same direction, momentum will be high.

In the middle, since the learning rate is already high, we should have have a high momentum, it will be too much.

at the end we can have higher momentum again, if the learning rate is too small, to speed up.

## Cross entropy

![](lesson5/lesson5-19.png)

It basically give a very small number if it predict confidently correctly.

It give a huge number if it predict confidently wrongly.

It give a somewhat high number if it is uncertain about its prediction.

```
D2=-B2*LOG(C2)-(1-B2)*LOG(1-C2)
```
A faster way is to do a lookup
```
E2=-IF(B2=1,LOG(C2),LOG(1-C2))
```
Note: The prediction output has to add up to 1. Or it will not work

E.g. pred(cat) + pred(dog) + pred(bear) = 1

E.g. pred(cat) + pred(not_cat) = 1

You can make sure it is 1, by using the correct activation at the end of your nerual network: softmax

## Softmax

![](lesson5/lesson5-20.png)
![](lesson5/lesson5-21.png)

the result of all the output is adding up to 1, and all are positive but smaller than 1.

First, calculate all the ouput with e to the power of that
Then, sum all output
Last, divide each with the sum